## KDP Computation
The notebook computes KDP using two methods (Maesaka and Giangrande's LP method) and write data to netcdf for analysis.


In [7]:
import pyart
import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import sys
import os.path as Path

#### First Function to run for KDP

In [8]:
def run_kdp_to_nc(radar_fname):
    try:
        radar = kdp(radar_fname)
    except OSError:
        print('ERROR: rerun file: '+radar_fname)
    else:
        write_kdp_to_nc(radar_fname, radar)

#### Function to compute KDP

In [9]:
def kdp(radar_fname):

    radar = read_sweep(radar_fname, sweep=3)    
    #### get texture
    #radar=texture(radar)
    PHIDP_LP, KDP_LP = pyart.correct.phase_proc_lp(radar, 0.0,
                                                   ncp_field='NCP',
                                                   refl_field='DBZ',
                                                   rhv_field='RHOHV',
                                                   phidp_field='PHIDP')

    radar.fields['PHIDP_LP']=PHIDP_LP
    radar.fields['KDP_LP']=KDP_LP
    
    kdp, _for_kdp, r_kdp = pyart.retrieve.kdp_maesaka(radar, refl_field='DBZ',
                                                  #psidp_field='PHIDP_LP')
    
    radar.fields['KDP_maesaka']=kdp
    
    return radar


In [10]:
def read_sweep(radar_fname, sweep=0):
    radar = pyart.io.read(radar_fname)
    radar = radar.extract_sweeps([sweep])
    return radar

#### The texture method is defined below but currently not used.

In [11]:
def texture(radar):
    nyquist_value = radar.fields['VEL']['data'].max()
    vel_texture = pyart.retrieve.calculate_velocity_texture(radar,
                                                        vel_field='VEL',
                                                        nyq=nyquist_value)

    radar.add_field('velocity_texture', vel_texture, replace_existing=True)

    phidp_texture = pyart.retrieve.texture_of_complex_phase(radar, phidp_field='PHIDP', phidp_texture_field='phidp_texture')

    radar.add_field('phidp_texture', phidp_texture, replace_existing=True)
    return radar

#### Write the selected data to file

In [12]:
def write_kdp_to_nc(radar_fname, radar):
    #remove unwanted fields
    radar.fields.pop('VEL')
    radar.fields.pop('WIDTH')
    radar.fields.pop('PHIDP')
    radar.fields.pop('unfolded_differential_phase')
    radar.fields.pop('DBZhv')
    
    cfrad_fname = Path.basename(radar_fname.replace('.b1.nc', '.b1_kdp-lp.nc'))
    cfrad_fname = '/ccsopen/home/braut/output/kdp_lp_cmac/'+cfrad_fname
    print('writing ... '+cfrad_fname)
    pyart.io.write_cfradial(cfrad_fname, radar)

# Main code

In [7]:
hour = '19'
day = '25'
month = '08'
year = '2022'
#files = sorted(glob.glob(f'/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/{year}{month}_glued/xprecipradar_guc_volume_{year}{month}{day}-{hour}*'))
#files = sorted(glob.glob(f'/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/{year}{month}_glued/xprecipradar_guc_volume_{year}{month}{day}*'))
files = sorted(glob.glob(f'/gpfs/wolf/atm124/proj-shared/gucxprecipradarcmacS2.c1/ppi/{year}{month}/*{year}{month}{day}*.nc'))

len(files)

240

In [10]:
for radar_fname in files:
    run_kdp_to_nc(radar_fname)



writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.000437.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.001037.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.001636.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.002237.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.002837.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.003437.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.004037.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.004637.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.005237.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac/gucxprecipradarcmacS2.c1.20220825.005837.nc
writing ... /ccsopen/home/braut/output/kdp_lp_cmac

# DASK

In [1]:
#radar = read_sweep(fname, sweep=3)
#fname =  '/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202208_glued/xprecipradar_guc_volume_20220825-042238.b1.nc'
#radar = pyart.io.read(fname)


from dask.distributed import Client, LocalCluster

cluster = LocalCluster()

cluster.scale(16)  # Sets the number of workers to 10
cluster.adapt(minimum=8, maximum=16)
client = Client(cluster)
#client

future = client.map(run_kdp_to_nc, files)


In [11]:
#my_data = client.gather(future)

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: Deprecation

glp_free: memory allocation error
Error detected in file env/alloc.c at line 70


2022-11-14 19:12:09,446 - distributed.nanny - WARNING - Restarting worker


glp_free: memory allocation error
Error detected in file env/alloc.c at line 70


2022-11-14 19:12:15,138 - distributed.nanny - WARNING - Restarting worker
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


glp_free: memory allocation error
Error detected in file env/alloc.c at line 70


ERROR 1: PROJ: proj_create_from_database: Open of /ccsopen/home/braut/analysis-env2/share/proj failed
2022-11-14 19:12:23,741 - distributed.nanny - WARNING - Restarting worker
ERROR 1: PROJ: proj_create_from_database: Open of /ccsopen/home/braut/analysis-env2/share/proj failed
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
ERROR 1: PROJ: proj_create_from_database: Open of /ccsopen/home/braut/analysis-env2/share/proj failed


glp_free: memory allocation error
Error detected in file env/alloc.c at line 70


2022-11-14 19:12:35,445 - distributed.worker - WARNING - Compute Failed
Key:       run_kdp_to_nc-febd6c305954c5b3b242bae218ea5257
Function:  run_kdp_to_nc
args:      ('/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202208_glued/xprecipradar_guc_volume_20220825-042238.b1.nc')
kwargs:    {}
Exception: "OSError(-51, 'NetCDF: Unknown file format')"



OSError: [Errno -51] NetCDF: Unknown file format: b'/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202208_glued/xprecipradar_guc_volume_20220825-042238.b1.nc'